In [2]:
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_erosion, binary_dilation
def otsu_threshold(hist, bin_centers):
    """
    Computes Otsu's threshold for a given histogram.
    
    Parameters:
    - hist (numpy.ndarray): Histogram counts.
    - bin_centers (numpy.ndarray): Centers of the histogram bins.
    
    Returns:
    - threshold (float): Computed threshold value.
    """
    total = hist.sum()
    current_max, threshold = 0, 0
    sum_total = np.dot(hist, bin_centers)
    sum_b, w_b = 0, 0
    
    for i in range(len(hist)):
        w_b += hist[i]
        if w_b == 0:
            continue
        w_f = total - w_b
        if w_f == 0:
            break
        sum_b += bin_centers[i] * hist[i]
        m_b = sum_b / w_b
        m_f = (sum_total - sum_b) / w_f
        # Between Class Variance
        var_between = w_b * w_f * (m_b - m_f) ** 2
        if var_between > current_max:
            current_max = var_between
            threshold = bin_centers[i]
    
    return threshold

def create_eroded_mask(input_path, output_path, threshold=None, erosion_iterations=1, erode=True):
    """
    Converts a NIfTI brain scan to a binary mask and erodes the mask.
    
    Parameters:
    - input_path (str): Path to the input NIfTI file.
    - output_path (str): Path to save the eroded mask NIfTI file.
    - threshold (float, optional): Intensity threshold for masking. If None, uses Otsu's method.
    - erosion_iterations (int): Number of times to apply erosion.
    
    Returns:
    - None
    """
    # Load the NIfTI image
    img = nib.load(input_path)
    data = img.get_fdata()
    
    # Step 1: Convert to Binary Mask
    if threshold is not None:
        print(f"Applying threshold: {threshold}")
        print(np.count_nonzero(data))
        mask = data > threshold
        print(np.count_nonzero(mask))
    else:
        print("Calculating Otsu's threshold.")
        from scipy import ndimage as ndi
        # Compute Otsu's threshold
        hist, bin_edges = np.histogram(data, bins=256, range=(np.min(data), np.max(data)))
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        threshold = otsu_threshold(hist, bin_centers)
        print(f"Computed Otsu's threshold: {threshold}")
        mask = data > threshold
    
    print(f"Initial mask has {np.sum(mask)} voxels.")
    
    # Step 2: Erode the Mask
    print(f"Eroding the mask with {erosion_iterations} iterations.")
    # Define a structuring element (3x3x3 cube)
    struct = np.ones((3, 3, 3), dtype=bool)
    if erode:
        eroded_mask = binary_erosion(mask, structure=struct, iterations=erosion_iterations)
    else:
        eroded_mask = binary_dilation(mask, structure=struct, iterations=erosion_iterations)
    if erosion_iterations==0:
        eroded_mask=mask
    
    print(f"Eroded mask has {np.sum(eroded_mask)} voxels.")
    
    # Step 3: Save the Eroded Mask
    # Convert boolean mask to float (1.0 and 0.0)
    eroded_mask_float = eroded_mask.astype(np.float32)
    
    # Create a new NIfTI image
    eroded_img = nib.Nifti1Image(eroded_mask_float, affine=img.affine, header=img.header)
    
    # Save the mask
    nib.save(eroded_img, output_path)
    print(f"Eroded mask saved to {output_path}")


In [ ]:
# Define file paths
input_nifti = "/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/example_patient/p00002.nii"
output_mask = "/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/example_patient/derivatives/p00002_brain_mask_xdil.nii"
erosion_iterations = 3 
threshold=1.1

Set erosion iterations to 0 if you only want to generate the mask of voxels passing threshold

In [ ]:
create_eroded_mask(input_nifti, output_mask, threshold=threshold, erosion_iterations=erosion_iterations, erode=False)